In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import requests

# Function to calculate values based on Put-Call Parity
def put_call_parity(S, K, r, T, C=None, P=None):
    present_value_of_strike = K * np.exp(-r * T)
    if C is None and P is not None:
        # Calculate Call value using Put-Call Parity
        C = P + S - present_value_of_strike
    elif P is None and C is not None:
        # Calculate Put value using Put-Call Parity
        P = C - S + present_value_of_strike
    elif C is None and P is None:
        raise ValueError("At least one of C (Call price) or P (Put price) must be provided.")
    return C, P, present_value_of_strike

# Function to identify arbitrage opportunities
def identify_arbitrage(S, K, r, T, C, P):
    """
    Identifies and explains arbitrage opportunities based on the put-call parity principle.

    Parameters:
    - S: Current stock price
    - K: Strike price
    - r: Risk-free interest rate
    - T: Time to maturity (in years)
    - C: Call option price
    - P: Put option price

    Returns:
    - Detailed explanation of the arbitrage strategy or a message indicating no arbitrage.
    """
    # Calculate the present value of the strike price and parity price
    _, _, pv_strike = put_call_parity(S, K, r, T, C, P)
    parity_price = P + S - pv_strike

    # Determine arbitrage opportunities
    if C > parity_price:
        return (
            f"Arbitrage Opportunity Detected (Overpriced Call):\n"
            f"Call Price (C): {C:.2f} > Parity Price: {parity_price:.2f}\n"
            f"Strategy:\n"
            f"1. Short the call option (Sell the call at {C:.2f}).\n"
            f"2. Buy the put option (Buy the put at {P:.2f}).\n"
            f"3. Buy the underlying asset (Stock price: {S:.2f}).\n"
            f"4. Borrow the present value of the strike price (PV(K): {pv_strike:.2f}) at the risk-free rate ({r:.2%}).\n"
            f"Outcome:\n"
            f"- Arbitrage profit locked in due to the mispricing of the call option."
        )
    elif C < parity_price:
        return (
            f"Arbitrage Opportunity Detected (Underpriced Call):\n"
            f"Call Price (C): {C:.2f} < Parity Price: {parity_price:.2f}\n"
            f"Strategy:\n"
            f"1. Buy the call option (Buy the call at {C:.2f}).\n"
            f"2. Short the put option (Sell the put at {P:.2f}).\n"
            f"3. Short the underlying asset (Stock price: {S:.2f}).\n"
            f"4. Invest the present value of the strike price (PV(K): {pv_strike:.2f}) at the risk-free rate ({r:.2%}).\n"
            f"Outcome:\n"
            f"- Arbitrage profit locked in due to the mispricing of the call option."
        )
    else:
        return (
            f"No Arbitrage Opportunity:\n"
            f"Call Price (C): {C:.2f} matches Parity Price: {parity_price:.2f}\n"
            f"The market prices are consistent with the put-call parity principle."
        )

# Function to fetch live stock price from Tiingo API
def fetch_live_price(ticker, token):
    url = f"https://api.tiingo.com/tiingo/daily/{ticker}/prices?token={token}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data[0]['close']
    else:
        raise ValueError(f"Failed to fetch data for ticker {ticker}. Check the ticker and token.")

# Function to plot profit/loss
def plot_profit(S, K, r, T, C, P):
    stock_prices = np.linspace(0.5 * K, 1.5 * K, 500)

    # Call and Put Payoffs
    call_payoff = np.maximum(stock_prices - K, 0) - C
    put_payoff = np.maximum(K - stock_prices, 0) - P

    # Combined Payoff for Arbitrage
    combined_payoff = call_payoff + put_payoff + stock_prices - K * np.exp(-r * T)

    # Plotting
    plt.figure(figsize=(5, 3))
    plt.plot(stock_prices, call_payoff, label="Call Payoff", linestyle="--")
    plt.plot(stock_prices, put_payoff, label="Put Payoff", linestyle="--")
    plt.plot(stock_prices, combined_payoff, label="Arbitrage Payoff", linewidth=2)
    plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
    plt.title("Profit/Loss Visualization")
    plt.xlabel("Stock Price at Expiration")
    plt.ylabel("Profit/Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to create a volatility heatmap
def plot_volatility_heatmap(K_values, T_values, volatility_matrix):
    plt.figure(figsize=(4, 4))
    sns.heatmap(volatility_matrix, annot=True, fmt=".2f", xticklabels=T_values, yticklabels=K_values, cmap="coolwarm")
    plt.title("Volatility Heatmap")
    plt.xlabel("Time to Expiry (T)")
    plt.ylabel("Strike Price (K)")
    plt.show()

# Interactive Widget
@widgets.interact
def interactive_tool(S=widgets.FloatText(value=100, description='S (Stock)'),
                     K=widgets.FloatText(value=100, description='K (Strike)'),
                     r=widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.005, description='r (Interest Rate)'),
                     T=widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='T (Time to Expiry)'),
                     C=widgets.FloatText(value=None, description='C (Call)'),
                     P=widgets.FloatText(value=None, description='P (Put)'),
                     ticker=widgets.Text(value='', description='Ticker'),
                     live_price=widgets.Checkbox(value=False, description='Use Live Price')):
    """
    Tool to explore put-call parity, arbitrage opportunities, profit/loss visualization, and volatility heatmaps.
    """
    try:
        # Fetch live stock price if enabled
        if live_price and ticker:
            S = fetch_live_price(ticker, "c8014c8227333b4647ff04d4378724f7345f3d4c")
            print(f"Live Stock Price for {ticker.upper()}: {S:.2f}")

        # Compute Call and Put values
        computed_C, computed_P, pv_strike = put_call_parity(S, K, r, T, C if C != 0 else None, P if P != 0 else None)

        # Display results
        print(f"\nComputed Values:")
        print(f"Call Price (C): {computed_C:.2f}")
        print(f"Put Price (P): {computed_P:.2f}")
        print(f"Present Value of Strike Price (K * e^(-rT)): {pv_strike:.2f}")

        # Identify Arbitrage
        print(f"\nArbitrage Analysis:")
        print(identify_arbitrage(S, K, r, T, C if C != 0 else computed_C, P if P != 0 else computed_P))

        # Plot Profit/Loss
        plot_profit(S, K, r, T, C if C != 0 else computed_C, P if P != 0 else computed_P)

        # Example Volatility Heatmap (using synthetic data for demonstration)
        K_values = np.linspace(80, 120, 5)
        T_values = np.linspace(0.5, 2, 5)
        volatility_matrix = np.random.rand(5, 5) * 0.5  # Synthetic volatility data
        plot_volatility_heatmap(K_values, T_values, volatility_matrix)
    except ValueError as e:
        print(f"Error: {e}")

interactive(children=(FloatText(value=100.0, description='S (Stock)'), FloatText(value=100.0, description='K (…